# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [ ]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 18.5MB/s]


### Import the required libraries

In [ ]:
import pandas as pd

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [ ]:
data = pd.read_csv('/content/ElectricCarData.csv')

In [ ]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

data['FastCharge_KmH'] = pd.to_numeric(data['FastCharge_KmH'], errors='ignore')
data['FastCharge_KmH'].fillna(value=10, inplace=True)

data['Brand'] = encoder.fit_transform(data['Brand'])
data['Model'] = encoder.fit_transform(data['Model'])
data['PowerTrain'] = encoder.fit_transform(data['PowerTrain'])
data['RapidCharge'] = encoder.fit_transform(data['RapidCharge'])
data['PlugType'] = encoder.fit_transform(data['PlugType'])
data['BodyStyle'] = encoder.fit_transform(data['BodyStyle'])
data['Segment'] = encoder.fit_transform(data['Segment'])

data

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,30,46,4.6,233,450,161,940.0,1,0,2,7,3,5,55480
1,31,33,10.0,160,270,167,250.0,1,2,2,1,2,5,30000
2,23,0,4.7,210,400,181,620.0,1,0,2,2,3,5,56440
3,2,101,6.8,180,360,206,560.0,1,2,2,6,3,5,68040
4,9,78,9.5,145,170,168,190.0,1,2,2,1,1,4,32997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,20,6,7.5,160,330,191,440.0,1,1,2,1,2,5,45000
99,1,94,4.5,210,335,258,540.0,1,0,2,6,4,5,96050
100,20,8,5.9,200,325,194,440.0,1,0,2,1,2,5,50000
101,20,10,5.1,200,375,232,450.0,1,0,2,1,2,5,65000


## Split the dataset for training and testing in ratio 80:20

In [ ]:
from sklearn.model_selection import train_test_split

data_copy = data.copy()

X_train, X_test, Y_train, Y_test = train_test_split(data_copy,  data['PriceEuro'], test_size=0.2)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [ ]:
from sklearn.tree import DecisionTreeRegressor
max_depth_value = 6
min_samples_split_value = 3
max_features_value = None

dt_regressor = DecisionTreeRegressor(
    max_depth=max_depth_value,
    min_samples_split=min_samples_split_value,
    max_features=max_features_value
)

dt_regressor.fit(X_train, Y_train)




DecisionTreeRegressor(max_depth=6, min_samples_split=3)

## Predict the outcomes for X test

In [ ]:
y_pred = dt_regressor.predict(X_test)
y_pred


array([ 45000.        ,  54575.83333333, 102945.        ,  67699.        ,
        31432.5       ,  60437.        ,  57500.        ,  37656.57142857,
        35638.25      ,  35638.25      ,  45000.        ,  37656.57142857,
        34276.66666667,  96520.        ,  41085.66666667,  24549.5       ,
       148650.5       ,  29612.4       ,  33004.4       , 102945.        ,
        75175.5       ])

## Assess the model performance, by using sklearn metrics for regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


y_pred = dt_regressor.predict(X_test)

mae_dt = mean_absolute_error(Y_test, y_pred)
mse_dt = mean_squared_error(Y_test, y_pred)
r2_dt = r2_score(Y_test, y_pred)

print(f"Mean Absolute Error: {mae_dt}")
print(f"Mean Squared Error: {mse_dt}")
print(f"R-squared: {r2_dt}")

Mean Absolute Error: 1540.5941043083901
Mean Squared Error: 6321535.427458698
R-squared: 0.993945837104595


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [ ]:
from xgboost import XGBRegressor

n_estimators_value = 105
max_depth_value = 2
learning_rate_value = 0.3
objective_value = 'reg:squarederror'

xgb_regressor = XGBRegressor(
    n_estimators=n_estimators_value,
    max_depth=max_depth_value,
    learning_rate=learning_rate_value,
    objective=objective_value
)

xgb_regressor.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=105, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [ ]:

y_pred = xgb_regressor.predict(X_test)
y_pred


array([ 42210.49 ,  53340.79 , 114879.12 ,  67459.32 ,  30226.832,
        59582.117,  59808.37 ,  37844.535,  34359.1  ,  34448.297,
        45369.074,  37303.36 ,  34127.496, 114914.65 ,  42059.05 ,
        23941.967, 148589.36 ,  27612.674,  33810.65 , 103587.4  ,
        68255.37 ], dtype=float32)

## Assess the model performance, by using sklearn metrics for regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae_xgb = mean_absolute_error(Y_test, y_pred)
mse_xgb = mean_squared_error(Y_test, y_pred)
r2_xgb = r2_score(Y_test, y_pred)

print(f"Mean Absolute Error: {mae_xgb}")
print(f"Mean Squared Error: {mse_xgb}")
print(f"R-squared: {r2_xgb}")



Mean Absolute Error: 2322.9332217261904
Mean Squared Error: 14788459.12441835
R-squared: 0.9858370262195526


## Compare the performances of both model for at least three regression metircs

In [ ]:
print("DecisionTreeRegressor Metrics:")
print(f"MAE: {mae_dt}")
print(f"MSE: {mse_dt}")
print(f"R-squared: {r2_dt}")
print("\nXGBRegressor Metrics:")
print(f"MAE: {mae_xgb}")
print(f"MSE: {mse_xgb}")
print(f"R-squared: {r2_xgb}")



DecisionTreeRegressor Metrics:
MAE: 1540.5941043083901
MSE: 6321535.427458698
R-squared: 0.993945837104595

XGBRegressor Metrics:
MAE: 2322.9332217261904
MSE: 14788459.12441835
R-squared: 0.9858370262195526
